# Task 1 (pen and paper)

In [1]:
layer1 = (224 * 224 * 3 * 100) + 100
print("Dense1:", layer1)
layer2 = (100 * 100) + 100
print("Dense2:", layer2)
layer3 = (100 * 10) + 10
print("Dense3:", layer3)

total = layer1 + layer2 + layer3
print("Total:", total)

Dense1: 15052900
Dense2: 10100
Dense3: 1010
Total: 15064010


# Task 2 (pen and paper)

In [2]:
conv1 = (5*5*3*32) + 32
print("Conv1:", conv1)
print("MaxPool:", 0)
conv2 = (5*5*32*32) + 32
print("Conv2:", conv2)
print("MaxPool:", 0)
print("Flatten:", 0)
dense1 = (512 * 100) + 100
print("Dense1:", dense1)
dense2 = (100 * 2) + 2
print("Dense2:", dense2)

total = conv1 + conv2 + dense1 + dense2
print("Total:", total)

Conv1: 2432
MaxPool: 0
Conv2: 25632
MaxPool: 0
Flatten: 0
Dense1: 51300
Dense2: 202
Total: 79566


In [3]:
# How many scalar multiplications take place on teh first convolutional layer?
n_mult = (64 * 64 * 32) * (5 * 5 * 3)
print("Number of scalar multiplications on the first layer:", n_mult)

Number of scalar multiplications on the first layer: 9830400


# Task 3 

In [4]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

# Task 4

In [6]:
import tensorflow as tf
base_model = tf.keras.applications.mobilenet.MobileNet(
    input_shape = (128,128,3),
    include_top = False,
    alpha = 0.25)

In [7]:
in_tensor = base_model.inputs[0] # Grab the input of base model
out_tensor = base_model.outputs[0] # Grab the output of base model


out_tensor = tf.keras.layers.Flatten()(out_tensor)

out_tensor = tf.keras.layers.Dense(units = 100, activation = "relu")(out_tensor)

out_tensor = tf.keras.layers.Dense(units = 10, activation = "softmax")(out_tensor)

# Define the full model by the endpoints.
model = tf.keras.models.Model(inputs = [in_tensor], outputs = [out_tensor])
#model.summary()

model.layers[-2].trainable = True
model.layers[-1].trainable = True

# Task 5 

In [14]:
from tensorflow.keras.backend import resize_images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(x_test.shape)

def resize(x, target_shape=(128, 128)):
    N, W, H = x.shape
    x_resized = [None] * N
    for i in range(x.shape[0]):
        x_resized[i] = cv2.resize(x[i], target_shape, interpolation=cv2.INTER_CUBIC)
    x_resized = np.array(x_resized)
    x_resized = x_resized[:, :, :, np.newaxis]
    x_resized = np.tile(x_resized, (1, 1, 1, 3))
    return x_resized
    
x_train_resized = resize(x_train)
x_test_resized = resize(x_test)

x_train_resized /= 255
x_test_resized /= 255

# convert class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

(60000, 28, 28)
(10000, 28, 28)


In [17]:
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train_resized, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,validation_data=(x_test_resized, y_test))

score = model.evaluate(x_test_resized, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 33s 558us/sample - loss: 0.1285 - accuracy: 0.9604 - val_loss: 0.1170 - val_accuracy: 0.9617
Epoch 2/12
60000/60000 [==============================] - 30s 495us/sample - loss: 0.1196 - accuracy: 0.9631 - val_loss: 0.1098 - val_accuracy: 0.9637
Epoch 3/12
60000/60000 [==============================] - 30s 499us/sample - loss: 0.1114 - accuracy: 0.9649 - val_loss: 0.1038 - val_accuracy: 0.9665
Epoch 4/12
60000/60000 [==============================] - 31s 517us/sample - loss: 0.1044 - accuracy: 0.9676 - val_loss: 0.0983 - val_accuracy: 0.9680
Epoch 5/12
60000/60000 [==============================] - 31s 517us/sample - loss: 0.0990 - accuracy: 0.9693 - val_loss: 0.0934 - val_accuracy: 0.9695
Epoch 6/12
60000/60000 [==============================] - 30s 508us/sample - loss: 0.0939 - accuracy: 0.9703 - val_loss: 0.0896 - val_accuracy: 0.9702
Epoch 7/12
60000/60000 [====================